# Capstone project - The battle of Neighbourhoods

This notebook is mainly for the Applied Data Science Capstone project

In [3]:
print('Hello Capstone Project Course!')
#!pip install geocoder

Hello Capstone Project Course!


## 1.Collecting data from website

In [5]:

import numpy as np
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = 'PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS'
CLIENT_SECRET = '1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2'
VERSION = '20180605'

In [6]:
url = 'https://en.wikipedia.org/wiki/Special_wards_of_Tokyo'

df = pd.read_html(url,header=0)
df_tokyo = df[3]
df_tokyo = df_tokyo.iloc[:23]
df_tokyo

,No.,Flag,Name,Kanji,Population(as of October 2016,Density(/km2),Area(km2),Major districts
0,01,NaN,Chiyoda,千代田区,59441,5100,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,..."
1,02,NaN,Chūō,中央区,147620,14460,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb..."
2,03,NaN,Minato,港区,248071,12180,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong..."
3,04,NaN,Shinjuku,新宿区,339211,18620,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich..."
4,05,NaN,Bunkyō,文京区,223389,19790,11.29,"Hongō, Yayoi, Hakusan"
5,06,NaN,Taitō,台東区,200486,19830,10.11,"Ueno, Asakusa"
6,07,NaN,Sumida,墨田区,260358,18910,13.77,"Kinshichō, Morishita, Ryōgoku"
7,08,NaN,Kōtō,江東区,502579,12510,40.16,"Kiba, Ariake, Kameido, Tōyōchō, Monzennakachō,..."
8,09,NaN,Shinagawa,品川区,392492,17180,22.84,"Shinagawa, Gotanda, Ōsaki, Hatanodai, Ōimachi,..."
9,10,NaN,Meguro,目黒区,280283,19110,14.67,"Meguro, Nakameguro, Jiyugaoka, Komaba, Aobadai"


Create a new dataframe and fill it in with data above

In [7]:
column_names = ['Name','Population','Latitude','Longitude']
column_2 = ['Name','District','Population','Latitude','Longitude']
tokyo = pd.DataFrame(columns = column_names)
tokyo_2 = pd.DataFrame(columns = column_2)

for name,pop,districts in zip(df_tokyo['Name'], df_tokyo['Population(as of October\xa02016'],df_tokyo['Major districts']):
    
    address = '{},Tokyo'.format(name)
    geolocator = Nominatim(user_agent="TK_explorer")
    location = geolocator.geocode(address)
    
    tokyo = tokyo.append({'Name':name,
                         'Population': pop,
                         'Latitude': location.latitude,
                         'Longitude': location.longitude}, ignore_index=True)
    districts_list = districts.split(',')
    for district in districts_list:
        tokyo_2 = tokyo_2.append({'Name':name,
                                 'District':district,
                                 'Population':pop,
                                 'Latitude': location.latitude,
                                 'Longitude': location.longitude},ignore_index=True)

In [11]:
tokyo.head()

,Name,Population,Latitude,Longitude
0,Chiyoda,59441,35.681091,139.767186
1,Chūō,147620,35.666255,139.775565
2,Minato,248071,35.644646,139.724120
3,Shinjuku,339211,35.692153,139.700300
4,Bunkyō,223389,35.705571,139.751970


Use the Foursquare to find venues for each special ward in Tokyo

In [12]:
radius = 500
LIMIT=100

neighborhood_latitude = 36.0611
neighborhood_longitude = 103.8343
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                     CLIENT_SECRET,
                                                                                                                     VERSION,
                                                                                                                     neighborhood_latitude,
                                                                                                                    neighborhood_longitude,
                                                                                                                        radius,
                                                                                                                            LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PKUTNEG4HV4S4KXTGXOJMW0OFPXR1ZNL3JZJUAHPIFM1E3BS&client_secret=1ANQ5XBNYQ3JA4OKCG2NRVNN0JBH1PNCV44Q5PAKH5TIJLR2&v=20180605&ll=36.0611,103.8343&radius=500&limit=100'

## Explore venues in Tokyo using Foursquare

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=800,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
LIMIT=100
Tokyo = getNearbyVenues(names=tokyo['Name'],
                                   latitudes=tokyo['Latitude'],
                                   longitudes=tokyo['Longitude']
                                  )

Chiyoda
Chūō
Minato
Shinjuku
Bunkyō
Taitō
Sumida
Kōtō
Shinagawa
Meguro
Ōta
Setagaya
Shibuya
Nakano
Suginami
Toshima
Kita
Arakawa
Itabashi
Nerima
Adachi
Katsushika
Edogawa


In [15]:
Tokyo

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chiyoda,35.681091,139.767186,Tokyo Station Hotel (東京ステーションホテル),35.681274,139.765917,Hotel
1,Chiyoda,35.681091,139.767186,Saryo Tsujiri (茶寮 都路里),35.681373,139.769289,Tea Room
2,Chiyoda,35.681091,139.767186,Fukuramu-chan (ふくらむちゃん柱),35.682498,139.766453,Exhibit
3,Chiyoda,35.681091,139.767186,KITTE Garden (屋上庭園 KITTEガーデン),35.679654,139.765169,Roof Deck
4,Chiyoda,35.681091,139.767186,Tokyo Station Gallery (東京ステーションギャラリー),35.682293,139.766332,Art Gallery
5,Chiyoda,35.681091,139.767186,Nemuro Hanamaru (回転寿司 根室花まる),35.679885,139.765150,Sushi Restaurant
6,Chiyoda,35.681091,139.767186,Honnoriya (ほんのり屋),35.680986,139.767774,Deli / Bodega
7,Chiyoda,35.681091,139.767186,Fruit Parlor Kajitsuen (フルーツパーラー 果実園),35.682280,139.767672,Café
8,Chiyoda,35.681091,139.767186,Prism Stone (プリズムストーン),35.682666,139.768764,Hobby Shop
9,Chiyoda,35.681091,139.767186,INTERMEDIATHEQUE (インターメディアテク),35.679988,139.765179,Museum


In [16]:
Tokyo.groupby('Name').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
Adachi,100,100,100,100,100,100
Arakawa,72,72,72,72,72,72
Bunkyō,100,100,100,100,100,100
Chiyoda,100,100,100,100,100,100
Chūō,100,100,100,100,100,100
Edogawa,35,35,35,35,35,35
Itabashi,35,35,35,35,35,35
Katsushika,32,32,32,32,32,32
Kita,100,100,100,100,100,100


In [17]:
print('There are {} uniques categories.'.format(len(Tokyo['Venue Category'].unique())))

There are 228 uniques categories.


## 3. Analyze each special ward

In [18]:
# one hot encoding
Tokyo_onehot = pd.get_dummies(Tokyo[['Venue Category']], prefix="", prefix_sep="")
Tokyo_onehot['Name'] = Tokyo['Name'] 

# move neighborhood column to the first column
fixed_columns = [Tokyo_onehot.columns[-1]] + list(Tokyo_onehot.columns[:-1])
Tokyo_onehot = Tokyo_onehot[fixed_columns]

Tokyo_onehot.head()

,Name,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wagashi Place,Wine Bar,Wine Shop,Yakitori Restaurant,Yoshoku Restaurant
0,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chiyoda,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
Tokyo_onehot.shape

(1841, 229)

In [20]:
Tokyo_grouped = Tokyo_onehot.groupby('Name').mean().reset_index()
Tokyo_grouped

,Name,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wagashi Place,Wine Bar,Wine Shop,Yakitori Restaurant,Yoshoku Restaurant
0,Adachi,0.00000,0.00,0.00,0.01,0.00,0.010000,0.010000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.01,0.00,0.010000,0.01,0.00,0.000000,0.020000
1,Arakawa,0.00000,0.00,0.00,0.00,0.00,0.000000,0.013889,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
2,Bunkyō,0.00000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.010000,0.00,0.00,0.000000,0.01,0.00,0.020000,0.010000
3,Chiyoda,0.00000,0.00,0.00,0.00,0.00,0.010000,0.010000,0.000000,0.000000,...,0.000000,0.020000,0.000000,0.01,0.00,0.010000,0.01,0.00,0.010000,0.000000
4,Chūō,0.00000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.020000
5,Edogawa,0.00000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
6,Itabashi,0.00000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
7,Katsushika,0.03125,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
8,Kita,0.00000,0.00,0.00,0.01,0.00,0.010000,0.010000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.01,0.00,0.010000,0.01,0.00,0.000000,0.020000
9,Kōtō,0.00000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000


In [21]:
Tokyo_grouped.shape

(23, 229)

 Print each Special Ward with its top 5 most common venues plus Ramen Restaurant

In [23]:
num_top_venues = 5

for name in Tokyo_grouped['Name']:
    print("----"+name+"----")
    temp = Tokyo_grouped[Tokyo_grouped['Name'] == name].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    output = temp.sort_values('freq', ascending=False).reset_index(drop=True)
    print(output[output['venue']=='Ramen Restaurant'])
    print(output.head(num_top_venues))

    print('\n')

----Adachi----
              venue  freq
4  Ramen Restaurant  0.04
                 venue  freq
0  Japanese Restaurant  0.07
1                Hotel  0.06
2                 Café  0.04
3           Hobby Shop  0.04
4     Ramen Restaurant  0.04


----Arakawa----
              venue  freq
4  Ramen Restaurant  0.04
                        venue  freq
0           Convenience Store  0.22
1               Grocery Store  0.08
2                        Park  0.06
3  Japanese Family Restaurant  0.04
4            Ramen Restaurant  0.04


----Bunkyō----
              venue  freq
0  Ramen Restaurant  0.09
              venue  freq
0  Ramen Restaurant  0.09
1          Sake Bar  0.05
2  Baseball Stadium  0.04
3             Hotel  0.04
4              Café  0.04


----Chiyoda----
              venue  freq
2  Ramen Restaurant  0.05
                     venue  freq
0      Japanese Restaurant  0.06
1                    Hotel  0.05
2         Ramen Restaurant  0.05
3              Coffee Shop  0.04
4  South Indi

In [166]:
def return_ramen_index_plus_most_common_venues(row, num_top_venues):
    #temp = Tokyo_grouped[Tokyo_grouped['Name'] == name].T.reset_index()
    temp = row.T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp_sorted = temp.sort_values('freq', ascending=False).reset_index(drop=True)
    Ramen_index=temp_sorted[temp_sorted['venue']=='Ramen Restaurant'].index.values.astype(int)[0]
    
    output=[]
    try:
        output.append('{}{} most common'.format(Ramen_index+1,indicators[Ramen_index%10]))
    except:
        output.append('{}th most common'.format(Ramen_index+1))
        
    for i in np.arange(num_top_venues):
        output.append(temp_sorted.iloc[i,0])
    
    return output

In [167]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name','Ramen Restaurant']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Tokyo_venues_sorted = pd.DataFrame(columns=columns)
Tokyo_venues_sorted['Name'] = Tokyo_grouped['Name']

for ind in np.arange(Tokyo_grouped.shape[0]):  
    Tokyo_venues_sorted.iloc[ind, 1:] = return_ramen_index_plus_most_common_venues(Tokyo_grouped.iloc[ind, :], num_top_venues)

Tokyo_venues_sorted.head()

,Name,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adachi,5th most common,Japanese Restaurant,Hotel,Café,Hobby Shop,Ramen Restaurant,Chocolate Shop,Deli / Bodega,Lounge,Tonkatsu Restaurant,Yoshoku Restaurant
1,Arakawa,5th most common,Convenience Store,Grocery Store,Park,Japanese Family Restaurant,Ramen Restaurant,Sake Bar,Italian Restaurant,Café,Tram Station,Noodle House
2,Bunkyō,1st most common,Ramen Restaurant,Sake Bar,Baseball Stadium,Hotel,Café,Japanese Restaurant,BBQ Joint,Bed & Breakfast,Italian Restaurant,Theater
3,Chiyoda,3rd most common,Japanese Restaurant,Hotel,Ramen Restaurant,Coffee Shop,South Indian Restaurant,Hobby Shop,Sushi Restaurant,Café,Dessert Shop,Tonkatsu Restaurant
4,Chūō,8th most common,Sushi Restaurant,Monjayaki Restaurant,Japanese Restaurant,Soba Restaurant,Italian Restaurant,Coffee Shop,Donburi Restaurant,Ramen Restaurant,Bakery,Tonkatsu Restaurant


## 4.Cluster Special Wards

Cluster the special wards into 5 clusters

In [172]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [173]:
kclusters = 4

Tokyo_grouped_clustering = Tokyo_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tokyo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 3, 1, 1, 0, 3], dtype=int32)

In [174]:
Tokyo_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tokyo_merged = tokyo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tokyo_merged = Tokyo_merged.join(Tokyo_venues_sorted.set_index('Name'), on='Name')

Tokyo_merged.head() # check the last columns!

,Name,Population,Latitude,Longitude,Cluster Labels,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chiyoda,59441,35.681091,139.767186,0,3rd most common,Japanese Restaurant,Hotel,Ramen Restaurant,Coffee Shop,South Indian Restaurant,Hobby Shop,Sushi Restaurant,Café,Dessert Shop,Tonkatsu Restaurant
1,Chūō,147620,35.666255,139.775565,0,8th most common,Sushi Restaurant,Monjayaki Restaurant,Japanese Restaurant,Soba Restaurant,Italian Restaurant,Coffee Shop,Donburi Restaurant,Ramen Restaurant,Bakery,Tonkatsu Restaurant
2,Minato,248071,35.644646,139.724120,0,6th most common,BBQ Joint,Japanese Restaurant,Café,French Restaurant,Sushi Restaurant,Ramen Restaurant,Italian Restaurant,Coffee Shop,Bakery,Yoshoku Restaurant
3,Shinjuku,339211,35.692153,139.700300,0,6th most common,BBQ Joint,Sake Bar,Bar,Café,Japanese Restaurant,Ramen Restaurant,Movie Theater,Bookstore,Pub,Coffee Shop
4,Bunkyō,223389,35.705571,139.751970,0,1st most common,Ramen Restaurant,Sake Bar,Baseball Stadium,Hotel,Café,Japanese Restaurant,BBQ Joint,Bed & Breakfast,Italian Restaurant,Theater


### Cluster 1

In [177]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 0, Tokyo_merged.columns[[0,1] + list(range(5, Tokyo_merged.shape[1]))]]

,Name,Population,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chiyoda,59441,3rd most common,Japanese Restaurant,Hotel,Ramen Restaurant,Coffee Shop,South Indian Restaurant,Hobby Shop,Sushi Restaurant,Café,Dessert Shop,Tonkatsu Restaurant
1,Chūō,147620,8th most common,Sushi Restaurant,Monjayaki Restaurant,Japanese Restaurant,Soba Restaurant,Italian Restaurant,Coffee Shop,Donburi Restaurant,Ramen Restaurant,Bakery,Tonkatsu Restaurant
2,Minato,248071,6th most common,BBQ Joint,Japanese Restaurant,Café,French Restaurant,Sushi Restaurant,Ramen Restaurant,Italian Restaurant,Coffee Shop,Bakery,Yoshoku Restaurant
3,Shinjuku,339211,6th most common,BBQ Joint,Sake Bar,Bar,Café,Japanese Restaurant,Ramen Restaurant,Movie Theater,Bookstore,Pub,Coffee Shop
4,Bunkyō,223389,1st most common,Ramen Restaurant,Sake Bar,Baseball Stadium,Hotel,Café,Japanese Restaurant,BBQ Joint,Bed & Breakfast,Italian Restaurant,Theater
5,Taitō,200486,5th most common,Coffee Shop,Hostel,BBQ Joint,Japanese Restaurant,Ramen Restaurant,Café,Dessert Shop,Sukiyaki Restaurant,Hotel,Yoshoku Restaurant
10,Ōta,722608,1st most common,Ramen Restaurant,Chinese Restaurant,Convenience Store,Sake Bar,Japanese Restaurant,Grocery Store,Tonkatsu Restaurant,Café,Dumpling Restaurant,Bed & Breakfast
12,Shibuya,227850,7th most common,Café,Record Shop,Sake Bar,Coffee Shop,BBQ Joint,Concert Hall,Ramen Restaurant,Bar,Nightclub,Japanese Restaurant
15,Toshima,294673,3rd most common,Japanese Restaurant,Hotel,Ramen Restaurant,Coffee Shop,South Indian Restaurant,Hobby Shop,Sushi Restaurant,Café,Dessert Shop,Tonkatsu Restaurant
16,Kita,345063,5th most common,Japanese Restaurant,Hotel,Café,Hobby Shop,Ramen Restaurant,Chocolate Shop,Deli / Bodega,Lounge,Tonkatsu Restaurant,Yoshoku Restaurant


### Cluster 2

In [178]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 1, Tokyo_merged.columns[[0,1] + list(range(5, Tokyo_merged.shape[1]))]]

,Name,Population,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Itabashi,569225,24th most common,Convenience Store,Park,Intersection,Bus Stop,Supermarket,Chinese Restaurant,Furniture / Home Store,Bowling Alley,Rest Area,Japanese Restaurant
21,Katsushika,447140,11st most common,Convenience Store,Grocery Store,Donburi Restaurant,Coffee Shop,Discount Store,Train Station,ATM,Bakery,Pharmacy,Ice Cream Shop


### Cluster 3

In [179]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 2, Tokyo_merged.columns[[0,1] + list(range(5, Tokyo_merged.shape[1]))]]

,Name,Population,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Sumida,260358,3rd most common,Convenience Store,Café,Ramen Restaurant,Park,Japanese Restaurant,Supermarket,Hotel,Soba Restaurant,Indian Restaurant,Coffee Shop
8,Shinagawa,392492,11st most common,Convenience Store,BBQ Joint,Furniture / Home Store,Pizza Place,Chinese Restaurant,Bus Stop,Racecourse,Supermarket,Sushi Restaurant,Soba Restaurant
9,Meguro,280283,41st most common,Convenience Store,Bus Stop,Grocery Store,Park,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Playground,Sushi Restaurant,Soba Restaurant
11,Setagaya,910868,10th most common,Convenience Store,Intersection,Café,Japanese Restaurant,Park,Bakery,Bus Stop,Diner,Tram Station,Ramen Restaurant
13,Nakano,332902,2nd most common,Convenience Store,Ramen Restaurant,Italian Restaurant,Café,Park,Grocery Store,BBQ Joint,Soba Restaurant,Noodle House,Diner
14,Suginami,570483,3rd most common,Park,Sake Bar,Ramen Restaurant,Convenience Store,Café,Chinese Restaurant,BBQ Joint,Grocery Store,Intersection,Dumpling Restaurant
17,Arakawa,213648,5th most common,Convenience Store,Grocery Store,Park,Japanese Family Restaurant,Ramen Restaurant,Sake Bar,Italian Restaurant,Café,Tram Station,Noodle House
19,Nerima,726748,7th most common,Theme Park Ride / Attraction,Convenience Store,Intersection,Grocery Store,Fishing Spot,Fast Food Restaurant,Ramen Restaurant,Café,Bookstore,Tennis Court


### Cluster 4

In [180]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 3, Tokyo_merged.columns[[0,1] + list(range(5, Tokyo_merged.shape[1]))]]

,Name,Population,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Kōtō,502579,17th most common,Intersection,Park,Convenience Store,Bus Stop,Rest Area,Dessert Shop,Pool,Baseball Field,Chinese Restaurant,Plaza
22,Edogawa,685899,6th most common,Convenience Store,Park,Intersection,Bus Stop,Baseball Field,Ramen Restaurant,Grocery Store,Gym,Music Venue,Chinese Restaurant


In [164]:
Tokyo_venues_sorted

,Name,Ramen Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adachi,5th most common,Japanese Restaurant,Hotel,Café,Hobby Shop,Ramen Restaurant,Chocolate Shop,Deli / Bodega,Lounge,Tonkatsu Restaurant,Yoshoku Restaurant
1,Arakawa,5th most common,Convenience Store,Grocery Store,Park,Japanese Family Restaurant,Ramen Restaurant,Sake Bar,Italian Restaurant,Café,Tram Station,Noodle House
2,Bunkyō,1st most common,Ramen Restaurant,Sake Bar,Baseball Stadium,Hotel,Café,Japanese Restaurant,BBQ Joint,Bed & Breakfast,Italian Restaurant,Theater
3,Chiyoda,3rd most common,Japanese Restaurant,Hotel,Ramen Restaurant,Coffee Shop,South Indian Restaurant,Hobby Shop,Sushi Restaurant,Café,Dessert Shop,Tonkatsu Restaurant
4,Chūō,8th most common,Sushi Restaurant,Monjayaki Restaurant,Japanese Restaurant,Soba Restaurant,Italian Restaurant,Coffee Shop,Donburi Restaurant,Ramen Restaurant,Bakery,Tonkatsu Restaurant
5,Edogawa,6th most common,Convenience Store,Park,Intersection,Bus Stop,Baseball Field,Ramen Restaurant,Grocery Store,Gym,Music Venue,Chinese Restaurant
6,Itabashi,24th most common,Convenience Store,Park,Intersection,Bus Stop,Supermarket,Chinese Restaurant,Furniture / Home Store,Bowling Alley,Rest Area,Japanese Restaurant
7,Katsushika,11st most common,Convenience Store,Grocery Store,Donburi Restaurant,Coffee Shop,Discount Store,Train Station,ATM,Bakery,Pharmacy,Ice Cream Shop
8,Kita,5th most common,Japanese Restaurant,Hotel,Café,Hobby Shop,Ramen Restaurant,Chocolate Shop,Deli / Bodega,Lounge,Tonkatsu Restaurant,Yoshoku Restaurant
9,Kōtō,17th most common,Intersection,Park,Convenience Store,Bus Stop,Rest Area,Dessert Shop,Pool,Baseball Field,Chinese Restaurant,Plaza


Visualize the clusters:

In [186]:
# create map
address = 'Tokyo'.format(name)
geolocator = Nominatim(user_agent="TK_explorer")
location = geolocator.geocode(address)
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tokyo_merged['Latitude'], Tokyo_merged['Longitude'], Tokyo_merged['Name'], Tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters